In [1]:
import pandas as pd
import snowflake.connector

from info import *
import numpy as np
import pandas as pd
from snow import *

#import pycaret
from pycaret.time_series import *


In [2]:
cnn = snowflake.connector.connect(

        user=snow_user,
        password=snow_password,
        account=snow_account,
        warehouse=snow_warehouse,
        database=snow_database)
sql ="""SELECT p.CODIGO_PAIS, e.ANIO, e.VALOR, i.DESCRIPCION as INDICADOR 
            FROM EV e 
            JOIN INDICADOR i 
            ON (e.ID_INDICADOR=i.ID_INDICADOR)
            JOIN PAIS p
            on (e.ID_PAIS=p.ID_PAIS)
            WHERE (e.ID_INDICADOR=18 OR e.ID_INDICADOR=31) AND e.ANIO>1960 AND e.ANIO<=2020
             """ 
datasql=pd.read_sql(sql,cnn)
data3=datasql.copy()

'USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA', 
    'MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','USA','IDN','RUS','TUR','ESP','BGR',
    'FRA','ITA','DEU','GBR','NOR','SWE','GRC'

In [35]:
var_target = ('EV_Total') # LABEL A USAR EN TARGET
var_exogena = ('Mortalidad_Infantil') # LABEL A USAR EN LA VARIABLE EXOGENA

FH=5 # Cantidad de periodos a predecir
Variacion_Hipotesis = -0.1 # MEJORA SETEADA EN LA HIPOTESIS (0.1 SIGNIFICA QUE MEJORA (SUBE) UN 10% ANUAL)

pais=['CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA', 
    'MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','USA','IDN','RUS','TUR','ESP','BGR',
    'FRA','ITA','DEU','GBR','NOR','SWE','GRC']

pais_lista=[]
ev_2020_lista=[]
ev_2025_lista=[]
ev_2025_mejora_lista=[]
modelo_mejora=[]

for i in pais:
    data_tar=data3[(data3['INDICADOR'] =='esperanza de vida al nacer, Total (años)')]
    data_exo=data3[(data3['INDICADOR'] =='Mortality rate, under-5 (per 1,000 live births)')]

    data2= pd.DataFrame()
    pais2=()
    pais2=(i)   
    data2=data_tar[(data_tar['CODIGO_PAIS'] ==pais2)]
    data2b=data_exo[(data_exo['CODIGO_PAIS'] ==pais2)]

    data2=data2.rename(columns={'VALOR':'EV_Total'})
    data2b=data2b.rename(columns={'VALOR':'Mortalidad_Infantil'})
    
    data2['date']=data2['ANIO']
    data2['date'] = pd.to_datetime(data2['date'],format='%Y')

    data2=data2.set_index('date')
    data2.drop(['CODIGO_PAIS','ANIO','INDICADOR'], axis = 1,inplace=True)

    data2b['date']=data2b['ANIO']
    data2b['date'] = pd.to_datetime(data2b['date'],format='%Y')

    data2b=data2b.set_index('date')
    #data2.drop(['CODIGO_PAIS','ANIO','INDICADOR'], axis = 1,inplace=True)
    data2['Mortalidad_Infantil']=data2b['Mortalidad_Infantil']
    data2copia = data2.copy()

    pais_lista.append(i)
    ev_2020_lista.append(data2[var_target].tail(1).values)

    target=data2[var_target]

    # SE ARMA UNA SERIE QUE EVOLUCIONA LA VAR EXOGENA CON LOS DATOS DE LA HIPOTESIS
    i = 1
    year2 = (2020 + i)
    while i < (FH + 1):

        new_mort=(data2.loc[data2.index[-1], var_exogena])*(1+Variacion_Hipotesis)
        new_row = {'index':year2, var_exogena:new_mort}
        data_time=(data2.tail(1).index[0]+pd.offsets.DateOffset(years=1))
        data2.loc[data_time]=new_row
        i+=1

    exo_hipot=pd.DataFrame(data2[var_exogena].tail(FH))

    # ENTRENO EL MODELO CON LA VARIABLE EXOGENA POR FH PERIODOS SIN EFECTO DE VAR EXOGENAS
  
    exp = TSForecastingExperiment()
    data_exo= pd.DataFrame(data2copia[var_exogena])
    s = setup(data_exo,target=var_exogena, fh = FH, fold=3, session_id = 123, verbose=False,scale_target='robust')
            
    best =  s.compare_models(n_select=3, turbo=True, verbose=False,exclude=['omp_cds_dt','knn_cds_dt'] )   # Cambiar a turbo False ,exclude=['omp_cds_dt','par_cds_dt']

    tuned_model = [s.tune_model(model, choose_better=True,n_iter=50 , verbose=False ) for model in best] # mejorar las iteraciones a 50

    blender = s.blend_models(tuned_model, choose_better=True, verbose=False )

    final_best1_exo = s.finalize_model(blender) # (setup sin var exogenas)

    df_fut1_exo=predict_model(final_best1_exo, fh = FH) # MODELO PREDICTOR DE LA VARIABLE EXOGENA

    # ENTRENO EL MODELO CON LA VARIABLE TARGET POR FH PERIODOS CON EFECTO DE VAR EXOGENAS
    exp = TSForecastingExperiment() 
    s2 = setup(data2copia,target=var_target, fh = FH, fold=3, session_id = 123, enforce_exogenous = True, verbose=False,scale_target='robust', scale_exogenous='robust') # enforce Exo True para forzar a q use solo modelos q acepten var exo
            
    best2 =  s2.compare_models(n_select=3, turbo=True, verbose=False, exclude=['omp_cds_dt','knn_cds_dt'] )   

    tuned_model2 = [s2.tune_model(model, choose_better=True,n_iter=50, verbose=False  ) for model in best2]

    blender2 = s2.blend_models(tuned_model2, choose_better=True, verbose=False )

    final_best1_target = s2.finalize_model(blender2)# MODELO PREDICTOR DE LA VARIABLE TARGET CON VAR EXOG

    modelo = pull()
    #modelo_mejora.append(modelo['Model'].values)
    #modelo_mejora.append(modelo.values)

    df_fut1_target_hipo=predict_model(final_best1_target,X=exo_hipot, fh = FH)#
    ev_2025_mejora_lista.append(df_fut1_target_hipo['y_pred'].tail(1).values)
    
    df_fut1_target_normal=predict_model(final_best1_target,X=df_fut1_exo, fh = FH)
    ev_2025_lista.append(df_fut1_target_normal['y_pred'].tail(1).values)

In [36]:
resultados = pd.DataFrame(pais_lista, columns=['Pais'])
resultados1 = pd.DataFrame(ev_2020_lista, columns=['EV_2020'])
resultados2 = pd.DataFrame(ev_2025_lista, columns=['EV_2025'])
resultados3 = pd.DataFrame(ev_2025_mejora_lista, columns=['EV_2025_Mejora'])
#resultados4 = pd.DataFrame(modelo_mejora, columns=['Modelo_Predictivo'])
resultado = pd.concat([resultados,resultados1,resultados2,resultados3],axis=1)#,resultados4
resultado.head(40)

,Pais,EV_2020,EV_2025,EV_2025_Mejora
0,CRI,80.46,81.1103,81.0872
1,PAN,78.68,79.5222,79.4800
2,BRA,76.08,77.4091,77.4056
3,ARG,76.81,77.4487,77.4446
4,CHL,80.33,80.8336,80.7986
5,URY,78.06,79.0204,79.0204
6,BOL,71.77,74.4941,74.4941
7,PER,76.95,77.7981,77.8208
8,EGY,72.15,72.9060,72.9063
9,LBY,73.08,73.6073,73.5602


In [39]:
resultado.to_csv('KPI_Mort-Inf.csv')